In [1]:
import geopandas as gpd
import numpy as np
from shapely.geometry import box

def create_grid(bounds, grid_size, crs):
    min_x, min_y, max_x, max_y = bounds
    cols = list(np.arange(min_x, max_x, grid_size))
    rows = list(np.arange(min_y, max_y, grid_size))
    polygons = []
    for x in cols:
        for y in rows:
            polygons.append(box(x, y, x+grid_size, y+grid_size))
    return gpd.GeoDataFrame({'geometry':polygons}, crs=crs)

# Load the GeoJSON file containing the polygons
data = gpd.read_file('spain.geojson')

# Define a projected CRS in meters (e.g., UTM)
projected_crs = 'EPSG:32630'  # UTM Zone 30N

# Project the original data to the projected CRS
data_projected = data.to_crs(projected_crs)

# Get the extent of Spain in the projected CRS
extent_projected = data_projected.total_bounds

# Define the grid size in meters
grid_size = 3000  # 30 km

# Create the grid
grid = create_grid(extent_projected, grid_size, projected_crs)

# Intersect the grid with the Spain polygon
grid_intersection = gpd.overlay(grid, data_projected, how='intersection')

# Save the grid as a GeoJSON file
grid_intersection.to_file('spain_grid_3km.geojson', driver='GeoJSON')
